Lab 06

In [215]:
# Name :- Arun M & Falak Ansari
# Reg No :- 23122110 & 23122106
# Date :- 04-04-2024

In [216]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
import pandas as pd

In [217]:
#Read the Smaller verison of the Dataset
df = pd.read_excel('copy.xlsx')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [218]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [219]:
#Increased the test size of the dataset
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.4,random_state=42)

In [220]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,female,38.0,1,0,71.2833,C
8,3,female,27.0,0,2,11.1333,S
4,3,male,35.0,0,0,8.0500,S
7,3,male,2.0,3,1,21.0750,S
3,1,female,35.0,1,0,53.1000,S
6,1,male,54.0,0,0,51.8625,S


In [221]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')
# fit the preprocessor to the training data
encoded_data=trf1.fit_transform(X_train)

#Printing the encoded data
print(encoded_data)


[[38.0 'C' 1 'female' 1 0 71.2833]
 [27.0 'S' 3 'female' 0 2 11.1333]
 [35.0 'S' 3 'male' 0 0 8.05]
 [2.0 'S' 3 'male' 3 1 21.075]
 [35.0 'S' 1 'female' 1 0 53.1]
 [54.0 'S' 1 'male' 0 0 51.8625]]


In [222]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

#Applied the OneHotEncoder to the dataframe
encoded_data2=trf2.fit_transform(X_train)

#Printed the encoded data  
print(encoded_data2)

[[ 1.      0.      1.      0.      1.     38.      1.      0.     71.2833]
 [ 1.      0.      0.      1.      3.     27.      0.      2.     11.1333]
 [ 0.      1.      0.      1.      3.     35.      0.      0.      8.05  ]
 [ 0.      1.      0.      1.      3.      2.      3.      1.     21.075 ]
 [ 1.      0.      0.      1.      1.     35.      1.      0.     53.1   ]
 [ 0.      1.      0.      1.      1.     54.      0.      0.     51.8625]]


In [223]:
# Scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])


In [224]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [225]:
clf = LogisticRegression()

In [226]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('clf',clf)
])

In [227]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001A25B6A4C10>)),
                ('clf', LogisticRegression())])

In [228]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [229]:
# Predict
y_pred = pipe.predict(X_test)

In [230]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)   

0.6

In [231]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

ValueError: n_splits=5 cannot be greater than the number of members in each class.

In [ ]:
# gridsearchcv
param_grid = {
    "clf__penalty": ['l1', 'l2'],
    "clf__C": [0.001, 0.01, 0.1, 1, 10, 100]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

C:\Users\arunp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\arunp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
36 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\arunp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8

ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

In [ ]:
grid.best_score_

nan

In [ ]:
grid.best_params_

{'clf__C': 0.001, 'clf__penalty': 'l1'}

In [ ]:
# export 
import pickle
pickle.dump(pipe,open('lr_pile.pkl','wb'))